In [16]:
import pandas as pd
import pymysql

# List of CSV files
csv_files = [
    "ap_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv",
    "himachal_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv",
    "up_bus_details.csv", "jk_bus_details.csv"
]

# Function to process each DataFrame
def process_dataframe(df, state_name):
    # If 'state' column doesn't exist, create it using the filename
    if 'state' not in df.columns:
        df['state'] = state_name
    return df

# Read and process CSV files into DataFrames
df_list = []
for file in csv_files:
    df = pd.read_csv(file)
    state_name = file.split('_')[0]  # Extract state name from filename
    df_list.append(process_dataframe(df, state_name))

# Concatenate all DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

# Ensure only one 'state' column exists
if 'state' in combined_df.columns:
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

# Add an 'id' column
combined_df['id'] = range(1, len(combined_df) + 1)

# Clean up 'Price' and 'Seat_Availability' columns
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')

# Drop any rows with missing values
combined_df = combined_df.dropna()

# Save combined DataFrame to CSV
combined_df.to_csv("bus_routes.csv", index=False)

# Connect to MySQL database
myconnection = pymysql.connect(host='localhost', user='root', passwd='', database="redbus")

# Prepare column definitions for MySQL table creation
column_definitions = ", ".join(
    f"`{col}` {dtype}".replace("float64", "FLOAT").replace("object", "TEXT").replace("int64", "INT")
    for col, dtype in zip(combined_df.columns, combined_df.dtypes)
)

# Define the table name
table_name = "bus_routes"

# Create the SQL table if it doesn't already exist
create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({column_definitions});"

try:
    with myconnection.cursor() as cursor:
        # Create table
        cursor.execute(create_table_query)
        
        # Insert data into the table
        placeholders = ", ".join(["%s"] * len(combined_df.columns))
        insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
        cursor.executemany(insert_query, combined_df.values.tolist())
    
    # Commit the transaction
    myconnection.commit()
    print("Data import completed successfully.")

except pymysql.Error as e:
    print(f"An error occurred: {e}")
    myconnection.rollback()

finally:
    myconnection.close()

<>:38: SyntaxWarning: invalid escape sequence '\d'
<>:38: SyntaxWarning: invalid escape sequence '\d'
C:\Users\SAM\AppData\Local\Temp\ipykernel_1864\2465646392.py:38: SyntaxWarning: invalid escape sequence '\d'
  combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')


An error occurred: (1060, "Duplicate column name 'state'")


In [18]:
import pandas as pd
import pymysql
import re

# List of CSV files
csv_files = [
    "ap_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv",
    "himachal_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "sb_bus_details.csv", "Telangana_bus_details.csv",
    "up_bus_details.csv", "jk_bus_details.csv"
]

# Function to process each DataFrame
def process_dataframe(df, state_name):
    # Rename columns to lowercase and replace spaces with underscores
    df.columns = [re.sub(r'\W+', '_', col.lower()) for col in df.columns]
    
    # If 'state' column doesn't exist, create it
    if 'state' not in df.columns:
        df['state'] = state_name
    return df

# Read and process CSV files into DataFrames
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        state_name = file.split('_')[0]
        df_list.append(process_dataframe(df, state_name))
        print(f"Successfully processed {file}")
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

# Concatenate all DataFrames
combined_df = pd.concat(df_list, ignore_index=True)

# Remove duplicate columns
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

# Add an 'id' column
combined_df['id'] = range(1, len(combined_df) + 1)

# Clean up 'price' and 'seat_availability' columns
combined_df['price'] = combined_df['price'].str.replace('INR ', '', regex=False).str.replace(',', '')
combined_df['seat_availability'] = combined_df['seat_availability'].str.extract(r'(\d+)', expand=False)

# Convert 'price' and 'seat_availability' to numeric
combined_df['price'] = pd.to_numeric(combined_df['price'], errors='coerce')
combined_df['seat_availability'] = pd.to_numeric(combined_df['seat_availability'], errors='coerce')

# Drop any rows with missing values
combined_df = combined_df.dropna()

print(f"Combined DataFrame shape: {combined_df.shape}")
print(f"Columns: {combined_df.columns.tolist()}")

# Save combined DataFrame to CSV
combined_df.to_csv("bus_routes.csv", index=False)
print("Saved combined data to bus_routes.csv")

try:
    # Connect to MySQL database
    myconnection = pymysql.connect(host='localhost', user='root', passwd='', database="redbus")
    print("Successfully connected to the database")

    # Prepare column definitions for MySQL table creation
    column_definitions = ", ".join(
        f"`{col}` {dtype}".replace("float64", "FLOAT").replace("object", "VARCHAR(255)").replace("int64", "INT")
        for col, dtype in zip(combined_df.columns, combined_df.dtypes)
    )

    # Define the table name
    table_name = "bus_routes"

    # Create the SQL table if it doesn't already exist
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({column_definitions});"

    with myconnection.cursor() as cursor:
        # Drop the table if it exists
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
        print(f"Dropped existing table '{table_name}'")

        # Create table
        cursor.execute(create_table_query)
        print(f"Table '{table_name}' created")
        
        # Insert data into the table
        placeholders = ", ".join(["%s"] * len(combined_df.columns))
        insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
        
        # Convert DataFrame to list of tuples for insertion
        data_to_insert = [tuple(x) for x in combined_df.to_numpy()]
        
        cursor.executemany(insert_query, data_to_insert)
        print(f"Inserted {cursor.rowcount} rows")
    
    # Commit the transaction
    myconnection.commit()
    print("Data import completed successfully")

except pymysql.Error as e:
    print(f"An error occurred: {e}")
    print(f"Error code: {e.args[0]}, Error message: {e.args[1]}")
    myconnection.rollback()

except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

finally:
    if 'myconnection' in locals() and myconnection.open:
        myconnection.close()
        print("Database connection closed")

print("Script execution completed")

Successfully processed ap_bus_details.csv
Successfully processed assam_bus_details.csv
Successfully processed chandigarh_bus_details.csv
Successfully processed himachal_bus_details.csv
Successfully processed kerala_bus_details.csv
Successfully processed rajasthan_bus_details.csv
Successfully processed sb_bus_details.csv
Successfully processed Telangana_bus_details.csv
Successfully processed up_bus_details.csv
Successfully processed jk_bus_details.csv
Combined DataFrame shape: (8980, 12)
Columns: ['route_name', 'route_link', 'bus_name', 'bus_type', 'departing_time', 'duration', 'reaching_time', 'star_rating', 'price', 'seat_availability', 'state', 'id']
Saved combined data to bus_routes.csv
Successfully connected to the database
Dropped existing table 'bus_routes'
Table 'bus_routes' created
Inserted 8980 rows
Data import completed successfully
Database connection closed
Script execution completed
